In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import random

In [3]:
# Load Frustration Dataset CSV
df = pd.read_csv('tracking_frustation_dataset_expanded.csv')

# Automatically determine input features
feature_columns = [
    'Reaction_Time', 'Cursor_Ball_Distance', 'Idle_Time', 'Error',
    'Error_Streak', 'Reaction_Variability', 'Recovery_Time',
    'Ball_Speed', 'Level_Difficulty_Index', 'Remaining_Level_Time', 'Target_Accuracy_Gap'
]
label_column = 'Frustration_Label'

# Automatically determine sequence length from most common group size
sequence_counts = df.groupby('Sequence_ID').size()
sequence_length = sequence_counts.mode()[0]

# Dataset Class for Frustration Detection
class FrustrationDataset(Dataset):
    def __init__(self, dataframe, sequence_length):
        self.sequence_length = sequence_length
        self.data = []
        self.labels = []

        grouped = dataframe.groupby('Sequence_ID')
        for _, group in grouped:
            group = group.sort_values('Timestep')
            features = group[feature_columns].values
            label = group[label_column].iloc[0]
            if len(features) == sequence_length:
                self.data.append(features)
                self.labels.append([label])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

# GRU Model Definition
class GRUFrustrationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUFrustrationModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return self.sigmoid(out)

# Fixed Hyperparameters
input_size = len(feature_columns)
hidden_size = 32
output_size = 1
batch_size = 8
epochs = 10
learning_rate = 0.005

# Dataset & DataLoader
dataset = FrustrationDataset(df, sequence_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, Optimizer
model = GRUFrustrationModel(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(epochs):
    total_loss = 0
    for sequences, labels in dataloader:
        outputs = model(sequences).squeeze()
        loss = criterion(outputs, labels.squeeze())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(dataloader):.4f}')

# Test Prediction on a Random Sequence
random_index = random.randint(0, len(dataset) - 1)
test_sequence = dataset[random_index][0].unsqueeze(0)
prediction = model(test_sequence).item()
print(f"\nTest Sequence Frustration Probability: {prediction:.4f}")


Epoch [1/10], Loss: 0.7121
Epoch [2/10], Loss: 0.6092
Epoch [3/10], Loss: 0.5352
Epoch [4/10], Loss: 0.4985
Epoch [5/10], Loss: 0.4604
Epoch [6/10], Loss: 0.4073
Epoch [7/10], Loss: 0.3734
Epoch [8/10], Loss: 0.2827
Epoch [9/10], Loss: 0.2418
Epoch [10/10], Loss: 0.1909

Test Sequence Frustration Probability: 0.8987
